In [1]:
# Take text file in sample.txt format and create an Excel 

import pandas as pd
import re
import json

'''
sample.txt
{
	"sheet_name": [
	{
		"c1": "10",
		"c2": "A10", 
		"c3": "W3"
		},
		{
		"c1": "20",
		"c2": "B10",
		"c3": "W4"
		}
	]
}
'''

file_name = 'sample.txt'

def txt_to_xlsx(input_file: str, output_file: str):
    # Read the text file content
    with open(input_file, 'r') as file:
        content = file.read()

    # Extract the first word in double quotes
    sheet_name = re.findall(r'"([^"]*)"', content)[0]

    # Load the JSON data from the content
    json_data = json.loads(content)

    # Convert the JSON data to a DataFrame
    df = pd.DataFrame(json_data[sheet_name])

    # Save the DataFrame as an xlsx file
    df.to_excel(output_file, sheet_name=sheet_name, index=False)

# Sample usage
txt_to_xlsx(file_name, file_name+'.xlsx')

In [2]:
# Generate image with QR code out of any url pasted into "url" variable

import qrcode
from PIL import Image

def generate_qr_code(url: str, output_file: str = "qr_code.png"):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(output_file)
    return Image.open(output_file)

url = "https://www.google.com/"
img = generate_qr_code(url)
img.show()

In [3]:
# Converts a json format into Excel with multiple sheets of data

import json
import pandas as pd

'''
sample2.json
{
	"start":"Begin",
	"end":"Finish",
	"source": [
	{
		"location": "10",
		"product": "A10", 
		"new": "W3"
		},
		{
		"location": "20",
		"product": "B10",
		"new": "W4"
		}
	],
	"coordinates":	[
	{
		"index": 1,
		"lat": -20.09,
		"long": null
		}
		]
}
'''

def json_to_excel(json_file_path, excel_file_path):
    # Load JSON data from file
    with open(json_file_path) as f:
        json_data = json.load(f)

    # Create dictionary of sheets and rows
    sheets = {}
    for table_name, table_data in json_data.items():
        for row in table_data:
            if table_name not in sheets:
                sheets[table_name] = []
            sheets[table_name].append(row)

    # Write data to Excel file
    with pd.ExcelWriter(excel_file_path) as writer:
        for sheet_name, sheet_data in sheets.items():
            df = pd.DataFrame(sheet_data)
            df.to_excel(writer, sheet_name=sheet_name, index=False)

file_name = 'sample2.json'
        
json_to_excel(file_name, 'output.xlsx')